In [3]:
from Crypto.Cipher import AES
import subprocess
import os
import numpy as np


In [2]:

def oracle(ciphertext):
    key = b'dont use the key'
    iv = b'ABCDEFGHabcdefgh'

    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext)
    #last byte tells us how much padding there is
    padnum = 16 - plaintext[-1]
    if padnum <= 0:
        return False
    passed_check = True
    for i in range(padnum):
        if plaintext[-i-1] != 16 - padnum:
            passed_check = False
            break
    return passed_check


In [68]:
def oracle(ciphertext):
    f = open("zna3tmp.cipher", "wb")
    f.write(ciphertext)
    f.close()
    res = subprocess.check_output('python oracle.py zna3tmp.cipher')
    return int(res.decode()[0]) == 1


In [4]:
f = open('ciphertext', "rb")
ciphertext = f.read()
f.close()

In [4]:
def decryptbyte(rs, yn, k):
    for i in range(k+1, 16):
        rs[i] ^= k
    
    for i in range(256):
        rs[k] = i
        guess = bytes(rs + yn)
        if oracle(guess):
            break
    
    if k == 15:

        for i in range(k):
            init = rs[i]
            for j in range(255):
                rs[i] = (init + j) % 256
                guess = bytes(rs + yn)
                if oracle(guess):
                    break
                else:
                    rs[k] ^= i

    rs[k] ^= k

    for i in range(k+1, 16):
        rs[i] ^= k

    return rs

In [5]:
def decryptblock(yn):
    rs = bytearray(list(np.random.randint(low=0, high=256, size=16)))
    for k in range(16):
        rs = decryptbyte(rs, yn, 15-k)
    return rs

In [8]:
def decrypt(ciphertext):
    size = len(ciphertext)
    if size <= 16:
        return

    xn = bytes()

    for i in range(size, 16, -16):
        yn = ciphertext[i-16:i]
        yn1 = ciphertext[i-32:i-16]
        dyn = decryptblock(yn)
        xn = bytes(a ^ b for a, b in zip(dyn, yn1)) + xn

    padnum = xn[-1]
    xn = xn[:padnum-16]
    xn
        
    return xn

In [9]:
decrypt(ciphertext)

b'a white rabbit with pink eyes'

In [26]:
f = open('ctext0', "r")
asd = f.read()
f.close()
res = bytearray([ int(a.strip()) for a in asd[1:-1].split(',')]).hex()
res


'ce5dfd5a8ff6a1c3bb4971c38e61a38754b7fa397c85a9f313e760910c0780e10698abd420d54d2b7ba172901590520d63939abf264bfe4dfd4f4f4298ce90baba83a457c6e4873e92f4858e835b2ed27e96af33b7c3d9c2b3d306862f7f8affa2e46036d683f2548c4fedcbddaac0f99f757e0a6417bd7a18e16eba121c50aa0718c399e9c8b5bb09c9628d8b02db2c9b1cac255077f58ad0cbb01c20fb47cd9819616cfe876263393548b8f2de2afddb4682eafe10fa2a3cc093fa2354221807d82520d824e101f78710cf81fb3f19890cd3af3640456d1f7b36c4f8ec523870a7f9223f6d50130fc512d0c0be1b00bc7c49a3ee9d16e23fb2a74e7eef9d38fffe951457495442ab3c7193506fb94236f7416ad6418dc5980e3dd1038b0915617774ddde84975d333ff14c0f0dfc8233ed1388fbdace4796be20428ea916665000082de8dc4589a405bd25a6937b252ed5f60398a620395c080702450fb408a127c9ad40796bcc9b2a7cfce2c1dcc39f7962be64985808b5349db6088c0578ab02a8daeaf210122cb14183d669ae502cd5ed75f90b408fdc4008e49fabd368efd32508e94f3ef5a102a55b80667db1ea6e5dd8e19df4f715b52ca433379f79cb66c68126548319e8346dcb442de735503c084de9e2581dcc5145fbefd67722fbff399e4f0a834d998f39194c0b8b2ae9da027

In [20]:
f = open('ctext1', "r")
asd = f.read()
f.close()
bytearray([ int(a.strip()) for a in asd[1:-1].split(',')]).hex()


'd747ae53cabea1d3f84160d38e25ee9041e3e72175c0fbf501fe2591190795e70798a4c124c7432b71e3319446de5e1463979fbc2853fe53fd5f53429ad2de87a985a457eaa5996ccfbbcb91dc0c338c3dc48972fcfbd3e1ffdd6dbb2b76d98fe581707580dbab1da942e7cbf8aac3a3cd2e6c4e3a01f23f56e420bf123f40a94514df99c8c5b8b4079762a5c518d1309618f864516cf9849591a25962a70ac3d7236765b2dd2b6325394eb8f5d339e09e42cae0fe10f92a308399b62452695005c46b35cd3af040ec91548789e634109848d7e30d46476d3b7b2fdcabf7542e70e1b418346d5c7e0ede1cd0e5b30755966d05a6bdc917b717b6a75375acdd74b4eed74c1f091873b17925b95866b9743ffc0624e75d9fc5981c3a8a01bb0816274e7bc9c69f8014247ed34e1814a7d606f11394b5889008cbe8311ed4a553544c0c5c23a7fe049aa15cb5698492736722c9a557bfad6c2e1b50505f0755a255e227eda1517b2dd4d96e3ec4f993cdc2cb23259c6f91016bb33887f34a990f64f803add6a4d6420334f249899f7ffc7c37c2b52cba5440db830311ac93f8ef64efcf2708d95522e9b102ac09c93454f8d92f41dafad9f8b658912da524299334df7bd6813306a058eb2162d14665ae6106684d01a7f71617836c47eabad2722ff7ba04d700178418b48a23054c57d17ffb984e2

In [23]:
asciis = list(range(32, 42)) + list(range(44, 60)) + [59, 63, 91, 93] + list(range(65, 91)) + list(range(97, 123))
bytearray(asciis)

bytearray(b' !"#$%&\'(),-./0123456789:;;?[]ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz')

In [5]:
from decrypt import decrypt, decryptblock
decrypt(ciphertext)

b'a white rabbit with pink eyes'

In [15]:
decrypt(b'ABCDEFGHabcdefghRa white rabbit wi')

b'Y!\x9c.\xc4\x7f\xe9yFi\x96p\xda\xdb\xf34'

In [12]:
len(b'ABCDEFGHabcdefghR\x9a^\x80\xb0\xa6\x07\xa9\t \xb2"U\x8fmo\xc2\xc3E\xd3\x8c\xbcY\x91\xcf=\xb9E\xe2\x0c\xc2\xdf')

47

In [6]:
decryptblock(b'ABCDEFGHabcdefgh')

bytearray(b'\x1a\xb8E\x01\xc9\x9c\x19,\xd2\x91i\x93\xb8\x02\xe7\xbb')

In [19]:
ptext = "ruo zhi er tong huan le duo, shabi?"


In [13]:
ptextbytes = bytearray(ptext.encode())
rounddown = int((len(ptextbytes))/16) * 16
diff = len(ptextbytes) - rounddown

for i in range(16 - diff):
    ptextbytes.append(diff)

ptextbytes

bytearray(b'ruo zhi er tong huan le duo, shabi?\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03')

In [15]:
ptextblocks = []
for i in range(0, len(ptextbytes), 16):
    ptextblocks.append(ptextbytes[i:i+16])
ptextblocks

[bytearray(b'ruo zhi er tong '),
 bytearray(b'huan le duo, sha'),
 bytearray(b'bi?\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03\x03')]

In [20]:
rs = bytearray(list(np.random.randint(low=0, high=256, size=16)))
c = [None for _ in range(len(ptextblocks))]
c[-1] = rs
c

[None,
 None,
 bytearray(b'4\xe9\x9e\xc3\x982@\x80\xbd\x93[\xe3\x90\xdc\x85\xd1')]

In [23]:
for i in range(len(ptextblocks)-1, 0, -1):
    c[i-1] = ptextblocks[i] ^ decryptblock(c[i])

iv = ptextblocks[0] ^ decryptblock(c[0])

iv, c

TypeError: unsupported operand type(s) for ^: 'bytes' and 'bytes'